# How to run REINVENT

Clone the reinvent-benchmarking github repo. More details found there.

In [1]:
#!git clone https://github.com/gkwt/reinvent-benchmarking.git

## Define fitness function from tartarus

Add the fitness function into the `custom.py` file of the repo. Default is the logP fitness function.

In [2]:
import sys, os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
ROOT_DIR = '..'
sys.path.append(ROOT_DIR)
sys.path.append('reinvent-benchmarking')

from tartarus import pce

def fitness_function(smi: str):
    dipole, hl_gap, lumo, obj, pce_1, pce_2, sas = pce.get_properties(smi)
    return pce_1 - sas

## Get the imports

In [3]:
#pip install seaborn

In [4]:
import os, sys
import pandas as pd
#import data_structs as ds
from data_structs_n import canonicalize_smiles_from_file, construct_vocabulary, write_smiles_to_file
from train_prior import pretrain
from train_agent import train_agent

2025-06-11 18:41:15.511227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749667275.526648  372205 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749667275.531558  372205 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749667275.543231  372205 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749667275.543247  372205 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749667275.543249  372205 computation_placer.cc:177] computation placer alr

## Prepare the dataset and vocabulary

In [5]:
#import data_structs
#print(data_structs.__file__)

In [6]:
data_path = os.path.join(ROOT_DIR, 'datasets')
print(data_path)
filename = 'Polyimides_synthetic.csv'  #'hce.csv'
sep = ','
header = 'infer'
smile_name = 'SMILES' #'smiles'

# dataset load
fname = os.path.join(data_path, filename)
data = pd.read_csv(fname, sep=sep, header=header)

headers =  ["SMILES", "Tg"]
data.columns = headers

data = data[headers][:50000]
print(data)

smiles = data[smile_name]

../datasets
                                                  SMILES     Tg
0      Ic1ccc(nc1)Cc1ccc2c(c1)cc(cc2)Cc1ccc(cn1)N1C(=...  508.0
1      O=C1c2cc(ccc2C(=O)N1c1ccc(c(c1Cl)Cl)S(=O)(=O)c...  641.0
2      Ic1cc(cc(c1)C(=O)O)C(=O)c1ccc2c(c1)ccc(c2)C(=O...  602.0
3      Clc1cc(cc(c1)C(C(F)(F)F)(C(F)(F)F)c1c(C)cc(cc1...  548.0
4      Cc1cc(Sc2ccc(nc2)Sc2cc(C)c(c(c2)C)I)cc(c1N1C(=...  502.0
...                                                  ...    ...
49995  Cc1cc(Cc2ccc(cc2)c2ccc(cc2)Cc2cc(C)c(c(c2)C)I)...  506.0
49996  Cc1cc(Cc2ccc(cc2)c2ccc(cc2)Cc2cc(C)c(c(c2)C)I)...  506.0
49997  Cc1cc(Cc2ccc(cc2)c2ccc(cc2)Cc2cc(C)c(c(c2)C)I)...  506.0
49998  Cc1cc(Cc2ccc(cc2)c2ccc(cc2)Cc2cc(C)c(c(c2)C)I)...  505.0
49999  Cc1cc(Cc2ccc(cc2)c2ccc(cc2)Cc2cc(C)c(c(c2)C)I)...  505.0

[50000 rows x 2 columns]


In [7]:
if not os.path.isdir('data'):
    os.mkdir('data')

# create smi file
with open(os.path.join('data', 'data.smi'), 'w') as f:
    for smi in smiles:
        f.write(smi+'\n')

smiles_file = 'data/data.smi'
print("Reading smiles...")
smiles_list = canonicalize_smiles_from_file(smiles_file)
print("Constructing vocabulary...")
voc_chars = construct_vocabulary(smiles_list)
write_smiles_to_file(smiles_list, "data/mols_filtered.smi")

Reading smiles...
0 lines processed.
5000 lines processed.
10000 lines processed.
15000 lines processed.
20000 lines processed.
25000 lines processed.
30000 lines processed.
35000 lines processed.
40000 lines processed.
45000 lines processed.
50000 SMILES retrieved
Constructing vocabulary...
Number of characters: 25


## Pretrain the network

In [16]:
num_epochs = 8 #100
verbose = False
train_ratio = 0.8
pretrain(num_epochs=num_epochs, verbose=verbose, train_ratio=train_ratio) #, restore_from = "data/Prior_PI_50k_epochs_8.ckpt"

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 44.46 GiB of which 832.00 KiB is free. Process 3554031 has 44.45 GiB memory in use. Of the allocated memory 1.44 GiB is allocated by PyTorch, and 10.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Start climbing algorithm for REINVENT

In [ ]:
from tartarus import docking

In [ ]:
train_agent(
    scoring_function='tanimoto', #'Tg_score'
    batch_size = 500,
    n_steps = 10
)

In [ ]:
# conda list python

In [ ]:
# %run ./reinvent-benchmarking/Pred_Tg/PI_Tg_Pred.py

In [ ]:
# pip install tf-keras

In [ ]:
#conda list sentence-transformers

In [ ]:
#pip install sentence-transformers==3.4.1

In [ ]:
#pip install sentence-transformers==3.4.1

In [ ]:
#pip show transformers

In [ ]:
#pip install transformers==4.28.1

In [14]:
#from Pred_Tg.PI_Tg_Pred import Pred_PI_Tg
import numpy as np
# Sous-échantillon du trainset pour équilibrer la visualisation, 50 k trop grand !
data_sample = data.sample(n=2000, random_state=42)
data_sample

,SMILES,Tg
33553,O=C(c1ccc2c(c1)C(=O)N(C2=O)I)c1ccc(cn1)C(c1ccc...,455.0
9427,Clc1cc(I)cc(c1)Sc1ccc(cc1C)Sc1cc(Cl)cc(c1)N1C(...,455.0
199,Ic1cc(cc(c1)C(F)(F)F)C(=O)c1ccc2c(c1)[nH]c1c2c...,515.0
12447,Cc1ccc(cc1Cc1cc(C)c(c(c1)C)c1c(C)cc(cc1C)Cc1cc...,523.0
39489,Ic1ccc(nc1)C(c1ccc2c(c1)ccc(c2)C(c1ccc(cn1)N1C...,466.0
...,...,...
33582,Ic1ccc(c(c1)Oc1cc(Oc2ccc(c(c2)N2C(=O)c3c(C2=O)...,479.0
31702,Ic1ccc(c(c1)C)Cc1ccc2c(c1)c1cc(ccc1C2(C)C)Cc1c...,510.0
38237,Ic1ccc(c(c1)C(C(F)(F)F)(C(F)(F)F)c1ccc(c(c1)Cl...,497.0
22358,Clc1cc(cc(c1)C(C(F)(F)F)(C(F)(F)F)c1c(C)cc(cc1...,548.0


In [11]:
#Evaluation de la performance du modele predictif
#data_sample['Tg_pred']=data_sample["SMILES"].apply(Pred_PI_Tg) #
#rmse = np.sqrt(np.mean((data_sample['Tg_pred'] - data_sample['Tg']) ** 2))
#mae = np.mean(np.abs(data_sample['Tg_pred'] - data_sample['Tg']))
#print(rmse, mae) #47.47540132795242 30.640225479125977

In [12]:
#from sklearn.metrics import r2_score
#r2 = r2_score(data_sample['Tg'], data_sample['Tg_pred'])
#print(r2) #0.30103430066668935

In [15]:
from Pred_Tg.PI_Tg_Pred import run_PolyBERT, finetune_PolyBERT

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

PI_generation = pd.read_csv("data/results/run_PI_50k_11_Juin_2025/results.csv")
print(PI_generation)

data_sample['PB']=data_sample["SMILES"].apply(finetune_PolyBERT) # (trainset)
data_sample['PB'] = data_sample['PB'].apply(eval)
data_sample['PB'] = data_sample['PB'].apply(np.array)

PI_generation['PB'] = PI_generation["smiles"].apply(finetune_PolyBERT) # un peu plus de 400 par generation, [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0] 
PI_generation['PB'] = PI_generation['PB'].apply(eval)
PI_generation['PB'] = PI_generation['PB'].apply(np.array)

# Liste des générations à tracer
generations = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

# Prépare la figure
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
axes = axes.flatten()

for idx, gen in enumerate(generations):
    ax = axes[idx]

    # Extraire les embeddings
    X_train = np.vstack(data_sample['PB'].values)
    X_gen = np.vstack(PI_generation[PI_generation['generation'] == gen]['PB'].values)

    # Créer un tableau combiné
    X = np.vstack([X_train, X_gen])
    labels = np.array(['train'] * len(X_train) + [f'gen_{int(gen)}'] * len(X_gen))

    # Projection t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    X_tsne = tsne.fit_transform(X)

    # Plot
    ax.scatter(X_tsne[labels == 'train', 0], X_tsne[labels == 'train', 1], alpha=0.3, label='Train', s=10)
    ax.scatter(X_tsne[labels != 'train', 0], X_tsne[labels != 'train', 1], alpha=0.7, label=f'Gen {int(gen)}', s=10)
    ax.set_title(f"t-SNE Projection: Generation {int(gen)}")
    ax.legend()
    print('ok')

plt.tight_layout()
plt.show()


                                                 smiles   fitness     score  \
0     Cc1cc(S(=O)(=O)c2ccc(S(=O)(=O)c3cc(C)c(N4C(=O)...  0.296010  0.214925   
1     Cc1cc(C(c2ccc(C(c3c(C)cc(I)cc3C)(C(F)(F)F)C(F)...  0.331492  0.330634   
2     O=c1c2cc3c(=O)n(-c4ccc(Oc5ccc(I)cn5)nc4)c(=O)c...  0.321152  0.297752   
3     Cc1ccc(C(c2ccc(C(c3cc(I)ccc3C)(C(F)(F)F)C(F)(F...  0.000000 -0.678468   
4     Cc1ccc(N2C(=O)c3cccc(-c4ccc5c(c4)C(=O)N(I)C5=O...  0.423963  0.584891   
...                                                 ...       ...       ...   
4737  Cc1c(I)cccc1Sc1ccc(Sc2cccc(-n3c(=O)c4cc5cc4c(=...  0.000000 -0.643401   
4738  Cc1cc(-c2cc(C)c(Sc3ccc(N4C(=O)c5ccc(-c6ccc7c(c...  0.386343  0.353131   
4739  Cc1c(Sc2ccc(N3C(=O)c4ccc(-c5ccc6c(c5)C(=O)N(I)...  0.000000 -0.643401   
4740  Cc1cc(I)cc(C)c1Sc1ccc(Cc2c(C)cc(N3C(=O)c4ccc(-...  0.385627  0.351291   
4741  Cc1cc(C(c2ccc(-c3ccc(C(c4ccc(-n5c(=O)cn6c(=O)c...  0.000000 -0.643401   

      generation  
0            0.0  
1            

NameError: name 'data_sample' is not defined